In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EvalPrediction
from datasets import Dataset
import re
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# Load your data
df = pd.read_csv("data.csv")
df['text'] = df['text'].apply(lambda x: re.sub(r"[^a-zA-ZığüşöçİĞÜŞÖÇ ]","",x).lower())
# Split into train and test sets
train_df, remaining_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, val_df = train_test_split(remaining_df, test_size=0.5, random_state=42)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased") # Use the smallest BERT model

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=512)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(val_df)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=len(set(df["label"]))) # Use the smallest BERT model


In [5]:

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    num_train_epochs=8,  # Adjust as needed
    per_device_train_batch_size=32,  # Adjust based on your GPU memory
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)


In [ ]:

# Define a function to compute metrics
def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset ,
    eval_dataset=tokenized_eval_dataset , # Using same dataset for eval in this example
    compute_metrics=compute_metrics, # Pass the metrics function to the Trainer
)

trainer.train()


In [ ]:

# Save the model
trainer.save_model("./best-model")

test_dataset = Dataset.from_pandas(test_df)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Evaluate on the test dataset
results = trainer.predict(tokenized_test_dataset)
predicted_labels = np.argmax(results.predictions, axis=1)
accuracy = accuracy_score(test_df["label"], predicted_labels)

print(f"Test Accuracy: {accuracy}")

In [ ]:
diff_df = test_df
diff_df['predicted'] = predicted_labels
diff_df = diff_df[diff_df['label'] != diff_df['predicted']]
diff_df

In [ ]:
diff_df.to_csv("false_predictions.csv",index=False)